In [1]:
import pandas as pd
import os
import re
import json

import nltk
nltk.download('punkt')

from nltk import sent_tokenize, word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hls\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# data 和 note
# NOTEEVENTS.csv Contains all notes for patients
note_csv_path = './data/original-csvs/NOTEEVENTS.csv'
train_path = './data/root1/train/'
test_path = './data/root/test/'
output_train_path = './data/output/train'
output_test_path = './data/output/test'

In [3]:
def mk_out_dir(outdir):
    try:
        os.makedirs(outdir)
    except:
        pass

In [4]:
mk_out_dir(output_train_path)
mk_out_dir(output_test_path)

# 取病人ID、入院ID、记录时间、以及内容文本都不为null的记录
没有HADM_ID的病人为门诊病人，本实验不考虑门诊病人

In [95]:
notes = pd.read_csv(note_csv_path)
notes.CHARTDATE = pd.to_datetime(notes.CHARTDATE)
notes.CHARTTIME = pd.to_datetime(notes.CHARTTIME)
notes.STORETIME = pd.to_datetime(notes.STORETIME)

mask =  notes.SUBJECT_ID.notnull() & notes.HADM_ID.notnull() & notes.CHARTTIME.notnull() & notes.TEXT.notnull()
notes_nonull = notes[mask]
notes_nonull = notes_nonull[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'TEXT']]
notes_nonull['HADM_ID'] = notes_nonull['HADM_ID'].astype(int)

In [96]:
notes_nonull.dtypes

SUBJECT_ID             int64
HADM_ID                int32
CHARTTIME     datetime64[ns]
TEXT                  object
dtype: object

In [97]:
notes_nonull

,SUBJECT_ID,HADM_ID,CHARTTIME,TEXT
308691,29075,179159,2116-02-07 14:08:00,"67M w/ h/o multiplemyeloma Dx [**2111**], neur..."
308692,18082,181163,2156-03-12 14:23:00,"[**Age over 90 52**]F with COPD on home O2, CA..."
308693,18082,181163,2156-03-12 14:28:00,"[**Age over 90 52**]F with COPD on home O2, CA..."
308694,16605,109285,2138-03-21 15:02:00,Chief Complaint:\n 24 Hour Events:\n Continu...
308695,29075,179159,2116-02-07 15:37:00,Chief Complaint:\n 24 Hour Events:\n EGD d...
...,...,...,...,...
2083175,31097,115637,2132-01-21 03:27:00,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,31097,115637,2132-01-21 09:50:00,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,31097,115637,2132-01-21 16:42:00,Family Meeting Note\nFamily meeting held with ...
2083178,31097,115637,2132-01-21 18:05:00,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [98]:
notes_nonull.groupby('HADM_ID').count()

,SUBJECT_ID,CHARTTIME,TEXT
HADM_ID,,,
100001,1,1,1
100003,23,23,23
100006,18,18,18
100007,7,7,7
100009,5,5,5
...,...,...,...
199993,71,71,71
199994,27,27,27
199995,20,20,20


In [99]:
print(notes_nonull.groupby('HADM_ID').count().describe())

         SUBJECT_ID     CHARTTIME          TEXT
count  55926.000000  55926.000000  55926.000000
mean      28.957283     28.957283     28.957283
std       59.891679     59.891679     59.891679
min        1.000000      1.000000      1.000000
25%        5.000000      5.000000      5.000000
50%       11.000000     11.000000     11.000000
75%       27.000000     27.000000     27.000000
max     1214.000000   1214.000000   1214.000000


In [100]:
del notes

# 处理notes

In [13]:
print(notes_nonull.iloc[0].TEXT)

67M w/ h/o multiplemyeloma Dx [**2111**], neuropathy, bed-bound, cared for by
   Dr [**Last Name (STitle) 731**] at [**Company 732**], last seen at [**Hospital1 54**] in [**2112**], a/w GIB. Pt was in
   USH at nursing home when maroon stools were noted by staff members.
   Patient himself was unaware of rectal bleeding. He denies GI symptoms.
   He reports slight lightheadedness. He was transferred from NH to [**Hospital1 **] ED
   where his HCT was 17.6 and plts 45, his NGT lavage was neg, he was
   given 2 units of PRBCs and 6 bags of plts.
   Gastrointestinal bleed, lower (Hematochezia, BRBPR, GI Bleed, GIB)
   Assessment:
   Pt required as additional 2 units of blood when he arrived in the MICU
   because his HCT only bumped to 19.8 after the 2 units if PRBCs that he
   received in the EW and an additional 2 bags of plt for a plt count of
   55.  He had an EGD [**2115-2-6**], no bleeding was noted.  He has had 4 liters
   of golytley and will have a colonoscopy [**2-7**].  His HCT

In [55]:
""" mask_3 = notes_nonull.SUBJECT_ID == 3
subject_3 = notes_nonull[mask_3]
re_str = subject_3.loc[1297688].TEXT
print(subject_3.shape[0])
print('=='*50)
print(subject_3.TEXT)

c_text = clean_text(re_str)
print(c_text)
if len(list(SECTION_TITLES.finditer(c_text))) == 0:
    print('vv')
for matcher in SECTION_TITLES.finditer(c_text):
    # add last
    # 得到匹配之前的去全部文本（并去掉空格）
    print(f'matcher: {matcher}') """

33
765519     [**2101-10-27**] 11:41 PM\n CHEST (PORTABLE AP...
765778     [**2101-10-26**] 6:01 AM\n CHEST (PORTABLE AP)...
765844     [**2101-10-24**] 8:05 AM\n CHEST (PORTABLE AP)...
765847     [**2101-10-25**] 6:32 AM\n CHEST (PORTABLE AP)...
765931     [**2101-10-24**] 4:06 PM\n CHEST (PORTABLE AP)...
765959     [**2101-10-22**] 4:27 PM\n CHEST (PORTABLE AP)...
765990     [**2101-10-20**] 10:23 PM\n CHEST (PORTABLE AP...
766104     [**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...
766114     [**2101-10-20**] 6:16 PM\n CHEST (PORTABLE AP)...
766194     [**2101-10-20**] 5:49 PM\n CT ABDOMEN W/O CONT...
766199     [**2101-10-21**] 4:43 PM\n CHEST (PORTABLE AP)...
766213     [**2101-10-23**] 5:22 AM\n CHEST (PORTABLE AP)...
766360     [**2101-10-21**] 6:10 AM\n CHEST (PORTABLE AP)...
766436     [**2101-10-21**] 1:00 AM\n CHEST (PORTABLE AP)...
1297688    Micu Progress Nursing Note:\n\nPatient arrived...
1297689    Respiratory Care Note:\n    Patient remains in...
1297690    MICU NPN 7

In [113]:
# re.I : 使得匹配对大小写不敏感
# re.M : 多行匹配，影响 ^ 和 $
SECTION_TITLES = re.compile(
    r'('
    r'ABDOMEN AND PELVIS|CLINICAL HISTORY|CLINICAL INDICATION|COMPARISON|COMPARISON STUDY DATE'
    r'|EXAM|EXAMINATION|FINDINGS|HISTORY|IMPRESSION|INDICATION'
    r'|MEDICAL CONDITION|PROCEDURE|REASON FOR EXAM|REASON FOR STUDY|REASON FOR THIS EXAMINATION'
    r'|TECHNIQUE'
    r'):|FINAL REPORT',
    re.I | re.M)

In [59]:
""" str = "this is string example....wow!!!"
print(len(str))
print(str.rjust(35, '0')) """

32
000this is string example....wow!!!


In [112]:
import re
import json

def split_heading(text):
    """Split the report into sections"""
    start = 0
    # finditer： 返回string中所有与pattern相匹配的全部字串，返回形式为迭代器
    if len(list(SECTION_TITLES.finditer(text))) == 0:
        yield text
    else:
        for matcher in SECTION_TITLES.finditer(text):
            # add last
            # 得到匹配之前的去全部文本（并去掉空格）
            print(f'matcher: {matcher}')
            end = matcher.start()
            #print(f'end: {end}')
            if end != start:
                section = text[start:end].strip()
                if section:
                    yield section

            # add title
            start = end
            end = matcher.end()
            if end != start:
                section = text[start:end].strip()
                if section:
                    yield section

            start = end

def find_end(text):
    """Find the end of the report."""
    ends = [len(text)]
    patterns = [
        re.compile(r'BY ELECTRONICALLY SIGNING THIS REPORT', re.I),
        re.compile(r'\n {3,}DR.', re.I),
        re.compile(r'[ ]{1,}RADLINE ', re.I),
        re.compile(r'.*electronically signed on', re.I),
        re.compile(r'M\[0KM\[0KM')
    ]
    for pattern in patterns:
        matchobj = pattern.search(text)
        if matchobj:
            ends.append(matchobj.start())
    return min(ends)

def pattern_repl(matchobj):
    """
    Return a replacement string to be used for match object
    """
    group_0 = matchobj.group(0)
    #print(f'group_0: {group_0}')
    # rjust默认填充为空格
    return ' '.rjust(len(group_0))

def clean_text(text):
    """
    Clean text:
         ①Replace [**Patterns**] with spaces.
         ②make sure the new text has the same length of old text.
    """

    # Replace [**Patterns**] with spaces.
    # re 模块提供了re.sub用于替换字符串中的匹配项, pattern_repl为替换项
    text = re.sub(r'\[\*\*.*?\*\*\]', pattern_repl, text)
    # Replace `_` with spaces.
    text = re.sub(r'_', ' ', text)

    start = 0
    
    end = find_end(text)
    new_text = ''
    if start > 0:
        new_text += ' ' * start
    new_text = text[start:end]

    # make sure the new text has the same length of old text.
    if len(text) - end > 0:
        new_text += ' ' * (len(text) - end)
    # 经过该函数处理后的text: clinical      doctor I have      a ...
    return new_text


def preprocess_mimic(text):
    """
    Preprocess reports in MIMIC-III.
    1. remove [**Patterns**] and signature
    2. split the report into sections
    3. tokenize sentences and words
    4. lowercase
    
    sent_tokenize: 句子切分器
    句子切分基本技术包括在句子之间寻找特定的分割符，例如句号(‘.’)，换行符(‘\n’)或者分号(‘;’)等；
    word_tokenize : 词切分器
    """

    for sec in split_heading(clean_text(text)):
        #print(f'preprocess_mimic sec: {sec}')
        for sent in sent_tokenize(sec):
            text = ' '.join(word_tokenize(sent))
            # 返回一个句子的字符串形式
            print(f'text: {text}')
            yield text.lower()

In [111]:
def all_files(file_path):
    all_files = os.listdir(file_path)
    all_folders = list(filter(lambda x: x.isdigit(), all_files))
    return all_folders


In [128]:
def preprocess_dataset(data_path, output_path):
    succeed = 0
    failed = 0
    sentence_lens = []
    hadm_id2index = {}

    allfiles = all_files(data_path)
    # find all records for every patients
    for pid in allfiles:
        print(f'Starting to process  patient_{pid} notes!')
        patient_id = int(pid)
        # find all notes for every patients
        sliced = notes_nonull[notes_nonull.SUBJECT_ID==patient_id]
        print(f'patient_{pid} have {sliced.shape[0]} recored!')
        if sliced.shape[0] == 0:
            print(f'No notes for patient: {patient_id}')
            failed += 1
            continue
        sliced.sort_values(by='CHARTTIME')

        # find all icustay records for every patients
        stays_path = os.path.join(data_path, pid, 'stays.csv')
        stays = pd.read_csv(stays_path)
        # get all hadm_ids for every patients
        hadm_ids = list(stays.HADM_ID.values)   
        print(f'patient_{pid} have {len(hadm_ids)} icu records')

        for i, hid in enumerate(hadm_ids):
            print(hid)
            print('--**--'*50)
            print(f'Starting to process "{i+1}" th notes of patient {pid}, hid: {hid}')
            # build hadm_id(hid) map to index ( order of the icustay records)
            hadm_id2index[str(hid)] = str(i)
            # get the "hid" note 
            sliced_hid = sliced[sliced.HADM_ID==hid]
            # text = sliced.TEXT.str.cat(sep=' ')
            if sliced_hid.shape[0] == 0:
                print(f'patient_{pid} have no record in hospital_{hid}')
                continue
            data_json = {}
            for index, row in sliced_hid.iterrows():
                # preprocess "TEXT"
                time = row["CHARTTIME"]
                text = row["TEXT"]
                # preprocess_mimic text
                processes_text = list(preprocess_mimic(text))
                print('==='*30)
                print(f'time: {time}; processes_text: {processes_text}')
                print('==='*30)
                data_json["{}".format(time)] = processes_text
            print('**'*50)
            #print(f'the {i} th data json (hid: {hid}): {data_json} of patient {pid} ')
            # save text
            """ outpath_pid = os.path.join(output_path, pid)
            try :
                os.makedirs(outpath_pid)
            except:
                pass  """
            out_path_pid = os.path.join(output_path, pid+'_'+str(i+1)+'.json')
            print('write json to file')
            with open(out_path_pid, 'w') as f:
                json.dump(data_json, f)
        succeed += 1

    print("Sucessfully Completed: %d/%d" % (succeed, len(allfiles)))
    print("No Notes for Patients: %d/%d" % (failed, len(allfiles)))
    # print("Failed with Exception: %d/%d" % (failed_exception, len(all_folders)))

    with open(os.path.join(output_path, 'train_hadm_id2index.json'), 'w') as f:
        json.dump(hadm_id2index, f)

In [127]:

notes_17 = notes_nonull[notes_nonull.SUBJECT_ID==17]
notes_161087 = notes_17[notes_17.HADM_ID==161087]
#print(notes_17)
print(notes_161087)

notes_23 = notes_nonull[notes_nonull.SUBJECT_ID==23]
notes_124321= notes_17[notes_17.HADM_ID==124321]
#print(notes_23)
print(notes_124321)  


         SUBJECT_ID  HADM_ID           CHARTTIME  \
1299670          17   161087 2135-05-09 17:04:00   
1299671          17   161087 2135-05-09 19:43:00   
1299875          17   161087 2135-05-10 05:31:00   

                                                      TEXT  
1299670  47yr female s/p pericardial window for tampona...  
1299671  CSRU Admission Note\n\nMrs. [**Known lastname ...  
1299875  Neuro: Pt. extubated at 2100 without incident....  
Empty DataFrame
Columns: [SUBJECT_ID, HADM_ID, CHARTTIME, TEXT]
Index: []


In [129]:
preprocess_dataset(train_path, output_train_path)
#preprocess_dataset(test_path, output_test_path)

Starting to process  patient_11 notes!
patient_11 have 16 recored!
patient_11 have 1 icu records
194540
--**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**----**--
Starting to process "1" th notes of patient 11, hid: 194540
matcher: <re.Match object; span=(309, 327), match='MEDICAL CONDITION:'>
text: 12:28 PM CT HEAD W/O CONTRAST Clip # Reason : please evaluate for any new changes Admitting Diagnosis : BRAIN MASS
text: MEDICAL CONDITION :
matcher: <re.Match object; span=(384, 412), match='REASON FOR THIS EXAMINATION:'>
text: 50 year old woman with brain mass .
text: Now has headache .
text: REASON FOR THIS EXAMINATION :
matcher: <re.Match object; span=(602, 614), match='FINAL REPORT'>
text: please evaluate for any new changes No contraindications fo

# a few tries

In [23]:
train_files = all_files(train_path)
# test_files = all_files(test_path)
train_files

['11', '17', '3', '4']

In [67]:
t1 = re.sub(r'\[\*\*.*?\*\*\]', pattern_repl, '[**Patterns**] and signature')
print(t1)

group_0: [**Patterns**]
               and signature


In [36]:
slice_3 = notes_nonull[notes_nonull.SUBJECT_ID==3]
#slice_3
slice_3.iloc[0].TEXT

'[**2101-10-27**] 11:41 PM\n CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 69440**]\n Reason: R/o MSK injury\n ______________________________________________________________________________\n [**Hospital 2**] MEDICAL CONDITION:\n  76 year old man with hypotension, vfib arrest, ? LLL\n  infiltrate/effusions, cough and reproducible chest pain bilaterally along\n  sternum\n REASON FOR THIS EXAMINATION:\n  R/o MSK injury\n ______________________________________________________________________________\n                                 FINAL REPORT\n INDICATION:  V-Fib arrest; ? infiltrates, with cough and bilateral chest pain\n along the sternum.  ? musculoskeletal injury.\n\n TECHNIQUE:  Single portable AP view of the chest is compared with [**2101-10-26**].\n\n FINDINGS:  There is an unchanged right-sided IJ catheter with the tip in the\n inferior SVC.  There is unchanged bilateral atelectasis and effusions, as well\n as cardiomegaly and

In [21]:
"""     
    pid = '1'
    i = 2
    outpath_pid = os.path.join(output_train_path, pid, str(i+1))
    try :
        os.makedirs(outpath_pid)
    except:
        pass  
"""